<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 21.6 MB/s eta 0:00:0

In [13]:
!pip install torch torchvision

In [14]:
!pip install transformers

In [15]:
import chromadb

In [16]:
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 37.9MiB/s]


{'ids': [['id2', 'id1']], 'distances': [[1.1462137699127197, 1.3015384674072266]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about oranges', 'This is a document about pineapple']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [17]:
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM, GenerationConfig)

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
config = GenerationConfig(max_new_tokens=200)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
def prompt(line: str) -> str:
  tokens = tokenizer(line, return_tensors="pt", return_token_type_ids=False)
  outputs = model.generate(**tokens)  # Remove generator_config
  result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  return result[0]

In [23]:
prompt("What is the capital of United States?")

'san francisco'

In [24]:
import collections
import plotly.express as px

counts = collections.defaultdict(int)

letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

texto = '''
Popular Linux distributions[16][17][18] include Debian, Fedora Linux, Arch Linux, and Ubuntu. Commercial distributions include Red Hat Enterprise Linux and SUSE Linux Enterprise. Desktop Linux distributions include a windowing system such as X11 or Wayland and a desktop environment such as GNOME, KDE Plasma or Xfce. Distributions intended for servers may not have a graphical user interface at all or include a solution stack such as LAMP. Many Linux distributions use the word "Linux" in their name, but the Free Software Foundation uses and recommends the name "GNU/Linux" to emphasize the use and importance of GNU software in many distributions, causing some controversy.[19][20] Because Linux is freely redistributable, anyone may create a distribution for any purpose.[21]

Linux was originally developed for personal computers based on the Intel x86 architecture, but has since been ported to more platforms than any other operating system.[22] Because of the dominance of Linux-based Android on smartphones, Linux, including Android, has the largest installed base of all general-purpose operating systems as of May 2022.[23][24][25] Linux is, as of March 2024, used by around 4 percent of desktop computers.[26] The Chromebook, which runs the Linux kernel-based ChromeOS, dominates the US K–12 education market and represents nearly 20 percent of sub-$300 notebook sales in the US.[27] Linux is the leading operating system on servers (over 96.4% of the top one million web servers' operating systems are Linux),[28] leads other big iron systems such as mainframe computers, and is used on all of the world's 500 fastest supercomputers[d] (as of November 2017, having gradually displaced all competitors).[29][30][31]
'''

texto  = texto.upper()

for letter in texto:
  if letter in letters:
    counts[letter] += 1

print(counts)
y = { letter: counts.get(letter, 0) for letter in letters }

print(y)

defaultdict(<class 'int'>, {'P': 36, 'O': 95, 'U': 69, 'L': 60, 'A': 100, 'R': 90, 'I': 94, 'N': 110, 'X': 20, 'D': 58, 'S': 113, 'T': 92, 'B': 27, 'C': 40, 'E': 149, 'F': 25, 'H': 37, 'M': 42, 'K': 10, 'W': 10, 'G': 18, 'Y': 21, 'V': 10, 'Z': 1})
{'A': 100, 'B': 27, 'C': 40, 'D': 58, 'E': 149, 'F': 25, 'G': 18, 'H': 37, 'I': 94, 'J': 0, 'K': 10, 'L': 60, 'M': 42, 'N': 110, 'O': 95, 'P': 36, 'Q': 0, 'R': 90, 'S': 113, 'T': 92, 'U': 69, 'V': 10, 'W': 10, 'X': 20, 'Y': 21, 'Z': 1}


In [25]:
fig = px.histogram(x=list(letters), y=y, title="Creature Names",
labels={"x": "Initial", "y": "Initial"})

In [11]:
fig